In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
###
tree_edges_table_name="tree_edges"
####
osm_nodes_table_name="osm_nodes"
osm_edges_table_name="osm_edges"
intersections_table_name="intersections"
2

2

In [3]:
DELETE_TABLES=True
if DELETE_TABLES:
    cmd="""DROP TABLE IF EXISTS `%s`"""%intersections_table_name
    cursor.execute(cmd)
    cmd="""DROP TABLE IF EXISTS `%s`"""%osm_edges_table_name
    cursor.execute(cmd)
    cmd="""DROP TABLE IF EXISTS `%s`"""%osm_nodes_table_name
    cursor.execute(cmd)

2

2

In [4]:
CREATE_TABLES=True
if CREATE_TABLES:
    cmd="""CREATE TABLE `%s`(
    `osm_id` BIGINT NOT NULL,
    `geometry` POINT NOT NULL,
    `elevation` FLOAT,

    PRIMARY KEY (osm_id)
    );
        """%osm_nodes_table_name
    
    cursor.execute(cmd)

    cmd="""CREATE TABLE `%s`(
    `osm_begin` BIGINT NOT NULL,
    `osm_end` BIGINT NOT NULL,
    `osm_key` INT NOT NULL,
    `length` FLOAT NOT NULL,
    `intersects` BOOLEAN,


    FOREIGN KEY (osm_begin) REFERENCES %s(osm_id),
    FOREIGN KEY (osm_end) REFERENCES %s(osm_id),
    PRIMARY KEY (osm_begin,osm_end,osm_key)

    );
        """%(osm_edges_table_name,osm_nodes_table_name,osm_nodes_table_name)
    
    cursor.execute(cmd)

    
    cmd="""CREATE TABLE `%s`(
        `id` INT NOT NULL AUTO_INCREMENT,
        `osm_begin` BIGINT NOT NULL,
        `osm_end` BIGINT NOT NULL,
        `osm_key` INT NOT NULL,
        `edge_coordinate` FLOAT NOT NULL,
        `geometry` POINT NOT NULL,
        `elevation` FLOAT NOT NULL,

        FOREIGN KEY (osm_begin,osm_end,osm_key) REFERENCES %s(osm_begin,osm_end,osm_key),
        UNIQUE (osm_begin,osm_end,osm_key,edge_coordinate),
        PRIMARY KEY (id)

          )
          ENGINE=InnoDB;
        """%(intersections_table_name,osm_edges_table_name)
    cursor.execute(cmd)
    

2

2

In [5]:
t1=time.time()
place_name='Haute Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='drive')
loops=[edge for edge in G_osm.edges() if edge[0]==edge[1]]
G_osm.remove_edges_from(loops)
G_osm=to_multi_graph(G_osm)
osm_crs=ox.graph_to_gdfs(G_osm,edges=False).estimate_utm_crs()
G_osm=ox.project_graph(G_osm,to_crs=osm_crs)
add_missing_geometries(G_osm)
t2=time.time()
print(t2-t1)
len(G_osm.nodes()),len(G_osm.edges())

32.464789152145386


(8551, 10201)

In [6]:
t1=time.time()
osm_edges_df=ox.graph_to_gdfs(G_osm,nodes=False,edges=True)
osm_edges_df['edge']=osm_edges_df.index
osm_edges_df=osm_edges_df.loc[:,['edge','geometry']]
osm_edges_df['number_of_points']=osm_edges_df['geometry'].apply(lambda ls:len(ls.coords))
osm_edges_df=osm_edges_df.set_index(np.arange(len(osm_edges_df)))
osm_edges_df['id_edge']=osm_edges_df.index
t2=time.time()
print(t2-t1)
osm_edges_df

0.18508195877075195


,edge,geometry,number_of_points,id_edge
0,"(14471679, 559203063, 0)","LINESTRING (513674.892 4668616.253, 513676.794...",115,0
1,"(14471679, 1249122279, 0)","LINESTRING (513674.892 4668616.253, 513670.837...",99,1
2,"(14471679, 9432824701, 0)","LINESTRING (513674.892 4668616.253, 513678.385...",960,2
3,"(559203063, 1249120583, 0)","LINESTRING (512760.729 4666834.201, 512751.915...",3,3
4,"(15901832, 1345257677, 0)","LINESTRING (543646.921 4669055.256, 543673.984...",2,4
...,...,...,...,...
10196,"(9025987889, 2219906091, 0)","LINESTRING (494959.779 4720020.262, 494962.220...",4,10196
10197,"(9081029075, 59744536, 0)","LINESTRING (494957.164 4720032.367, 494953.329...",4,10197
10198,"(9081029075, 14471820, 0)","LINESTRING (494957.164 4720032.367, 494956.752...",3,10198
10199,"(9744995304, 9744995311, 0)","LINESTRING (496541.756 4720170.475, 496521.473...",2,10199


In [7]:
t1=time.time()
intersection=increasing_elevations_intersections(osm_edges_df,osm_crs,cursor)
t2=time.time()
print('total : %f'%(t2-t1))

0.000000<=elevation<50.000000 :4.761411 
50.000000<=elevation<100.000000 :5.308592 
100.000000<=elevation<150.000000 :5.101337 
150.000000<=elevation<200.000000 :5.035397 
200.000000<=elevation<250.000000 :5.492108 
250.000000<=elevation<300.000000 :5.449070 
300.000000<=elevation<350.000000 :7.481649 
350.000000<=elevation<400.000000 :6.359187 
400.000000<=elevation<450.000000 :6.477753 
450.000000<=elevation<500.000000 :6.301240 
500.000000<=elevation<550.000000 :5.773319 
550.000000<=elevation<600.000000 :6.425088 
600.000000<=elevation<650.000000 :7.297439 
650.000000<=elevation<700.000000 :7.103457 
700.000000<=elevation<750.000000 :6.458812 
750.000000<=elevation<800.000000 :7.075906 
800.000000<=elevation<850.000000 :5.050310 
850.000000<=elevation<900.000000 :4.359910 
900.000000<=elevation<950.000000 :4.003208 
950.000000<=elevation<1000.000000 :3.793034 
1000.000000<=elevation<1050.000000 :3.455006 
1050.000000<=elevation<1100.000000 :3.224403 
1100.000000<=elevation<1150.000

In [8]:
len(intersection),len(intersection.drop_duplicates(['edge','edge_coordinate']))

(48664, 45447)

In [9]:
t1=time.time()
bad_indexes=[]
for _,df in intersection.groupby(['edge','edge_coordinate']):
    if len(df)>1:
        if len(df['elevation'].unique())>1:
            print('c est la merde')
        else:
            bad_indexes+=list(df.index[1:])
intersection=intersection.drop(labels=bad_indexes)
t2=time.time()
print(t2-t1)


33.65165901184082


In [10]:
bad_indexes

[48209,
 48212,
 48219,
 48221,
 48223,
 48227,
 48319,
 48321,
 48323,
 48325,
 48327,
 48335,
 48342,
 48341,
 48338,
 48337,
 48445,
 48447,
 48448,
 48446,
 48344,
 48331,
 48328,
 48320,
 48224,
 48222,
 48218,
 48208,
 5197,
 36828,
 44902,
 44898,
 44897,
 44896,
 44895,
 44894,
 44701,
 44702,
 46209,
 46210,
 46211,
 46214,
 45055,
 45056,
 45057,
 45042,
 44710,
 44711,
 44713,
 44715,
 46120,
 46121,
 46122,
 46123,
 46126,
 44666,
 44669,
 44670,
 44667,
 44671,
 44672,
 44668,
 44658,
 43531,
 45086,
 43375,
 43373,
 43372,
 43370,
 43371,
 43367,
 43369,
 43366,
 43364,
 43361,
 43360,
 43359,
 42230,
 42229,
 45080,
 45081,
 45077,
 45075,
 45076,
 45079,
 45036,
 42778,
 42776,
 42774,
 42773,
 42769,
 40933,
 40930,
 40929,
 40926,
 40921,
 40918,
 40920,
 45034,
 42758,
 42756,
 42755,
 42754,
 42751,
 42750,
 42749,
 40898,
 40896,
 40893,
 40892,
 40891,
 40890,
 26893,
 26895,
 26896,
 26897,
 26898,
 29144,
 29145,
 29148,
 30662,
 30666,
 30668,
 34217,
 34216,
 

In [10]:
(48664, 45442)

(48664, 45442)

In [11]:
(42890, 42890)

(42890, 42890)

In [12]:
ADD_OSM_DATA=True
if ADD_OSM_DATA:
    t1=time.time()
    edges=osm_edges_df['edge']
    osm_nodes_data=nx.edge_subgraph(G_osm,edges).nodes(data=True)
    
    string_list=',\n'.join(['(%i,ST_GeomFromText(\'%s\'))'%(node,Point(data['lon'],data['lat']).wkt) for node,data in osm_nodes_data])
    cmd="INSERT IGNORE INTO %s(osm_id,geometry) VALUES %s;"%(osm_nodes_table_name,string_list)
    cursor.execute(cmd)

    intersecting_edges=intersection['edge'].unique()
    string_list=',\n'.join(['(%i,%i,%i,%f,TRUE)'%(edge[0],edge[1],edge[2],G_osm.get_edge_data(*edge)['length']) for edge in intersecting_edges])
    cmd="INSERT INTO %s(osm_begin,osm_end,osm_key,length,intersects) VALUES %s;"%(osm_edges_table_name,string_list)
    cursor.execute(cmd)

    inside_edges=set(edges).difference(intersecting_edges)

    string_list=',\n'.join(['(%i,%i,%i,%f,False)'%(edge[0],edge[1],edge[2],G_osm.get_edge_data(*edge)['length']) for edge in inside_edges])
    cmd="INSERT INTO %s(osm_begin,osm_end,osm_key,length,intersects) VALUES %s;"%(osm_edges_table_name,string_list)
    cursor.execute(cmd)


    ###


    ###SOLVE IGNORE




    ###
    intersection=intersection.to_crs('epsg:4326')
#    step=10000
#    for k in range(0,len(intersection)):
#        sub_intersection=intersecion.iloc[k:min(len(intersection,k+step)]
    string_list=',\n'.join(['(%i,%i,%i,%f,%f,ST_GeomFromText(\'%s\'))'%(row['edge'][0],row['edge'][1],row['edge'][2],row['edge_coordinate'],row['elevation'],row['geometry'].wkt)  for _,row in intersection.iterrows()])
    cmd="INSERT INTO %s (`osm_begin`,`osm_end`,`osm_key`,`edge_coordinate`,`elevation`,`geometry`) VALUES %s;"%(intersections_table_name,string_list)
    cursor.execute(cmd)
    
    t2=time.time()
    print(t2-t1)
2

IntegrityError: 1062 (23000): Duplicate entry '2360730252-6942635364-0-891.229' for key 'intersections.osm_begin'